In [4]:
import sqlite3
import pandas as pd

con = sqlite3.connect("../output/lgbm/opt/db/optimization_lgbm.db")

# List all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con)
print(tables)

# Read a specific table (for example, optuna studies)
df = pd.read_sql_query("SELECT * FROM trial_params WHERE trial_id = 1;", con)
print(df.head())

con.close()


                         name
0                     studies
1                version_info
2            study_directions
3       study_user_attributes
4     study_system_attributes
5                      trials
6       trial_user_attributes
7     trial_system_attributes
8                trial_params
9                trial_values
10  trial_intermediate_values
11           trial_heartbeats
12            alembic_version
   param_id  trial_id        param_name  param_value  \
0         5         1  bagging_fraction     0.984823   
1         4         1  feature_fraction     0.901683   
2         2         1     learning_rate     0.262822   
3         3         1  min_data_in_leaf   295.000000   
4         1         1        num_leaves    83.000000   

                                   distribution_json  
0  {"name": "FloatDistribution", "attributes": {"...  
1  {"name": "FloatDistribution", "attributes": {"...  
2  {"name": "FloatDistribution", "attributes": {"...  
3  {"name": "IntDistrib

In [ ]:
import polars as pl

In [8]:
df = pl.DataFrame({
    "foto_mes": [202101, 202102, 202103, 202104, 202105, 202106],
    "clase_ternaria": ["BAJA+1", "BAJA+2", "CONTINUA", "BAJA+2", "BAJA+1", "BAJA+2"]
})

df = df.with_columns(
    pl.lit(1.0).alias("clase_peso")
)

df = df.with_columns(clase_peso_2 = pl.lit(1.0))

df

foto_mes,clase_ternaria,clase_peso,clase_peso_2
i64,str,f64,f64
202101,"""BAJA+1""",1.0,1.0
202102,"""BAJA+2""",1.0,1.0
202103,"""CONTINUA""",1.0,1.0
202104,"""BAJA+2""",1.0,1.0
202105,"""BAJA+1""",1.0,1.0
202106,"""BAJA+2""",1.0,1.0


In [9]:
df = df.with_columns(
    pl.when(pl.col("clase_ternaria") == "BAJA+2").then(1.00002)
    .when(pl.col("clase_ternaria") == "BAJA+1").then(1.00001)
    .otherwise(pl.col("clase_peso"))
    .alias("clase_peso")
)

df

foto_mes,clase_ternaria,clase_peso,clase_peso_2
i64,str,f64,f64
202101,"""BAJA+1""",1.00001,1.0
202102,"""BAJA+2""",1.00002,1.0
202103,"""CONTINUA""",1.0,1.0
202104,"""BAJA+2""",1.00002,1.0
202105,"""BAJA+1""",1.00001,1.0
202106,"""BAJA+2""",1.00002,1.0


In [10]:
df = df.with_columns(
    pl.when(pl.col("clase_ternaria") == "CONTINUA").then(pl.lit(0))
    .otherwise(pl.lit(1))
)

df

foto_mes,clase_ternaria,clase_peso,clase_peso_2,literal
i64,str,f64,f64,i32
202101,"""BAJA+1""",1.00001,1.0,1
202102,"""BAJA+2""",1.00002,1.0,1
202103,"""CONTINUA""",1.0,1.0,0
202104,"""BAJA+2""",1.00002,1.0,1
202105,"""BAJA+1""",1.00001,1.0,1
202106,"""BAJA+2""",1.00002,1.0,1


In [11]:
train_data = df.filter(pl.col("foto_mes").is_in([202101,202102, 202103]))
test_data = df.filter(pl.col("foto_mes") == 202106)

In [12]:
train_data

foto_mes,clase_ternaria,clase_peso,clase_peso_2,literal
i64,str,f64,f64,i32
202101,"""BAJA+1""",1.00001,1.0,1
202102,"""BAJA+2""",1.00002,1.0,1
202103,"""CONTINUA""",1.0,1.0,0


In [13]:
test_data

foto_mes,clase_ternaria,clase_peso,clase_peso_2,literal
i64,str,f64,f64,i32
202106,"""BAJA+2""",1.00002,1.0,1


In [14]:
X_train = train_data.drop(['clase_ternaria', 'clase_peso'])
X_train

foto_mes,clase_peso_2,literal
i64,f64,i32
202101,1.0,1
202102,1.0,1
202103,1.0,0


In [16]:
y_train_binaria = train_data['clase_peso'].to_numpy()
y_train_binaria

array([1.00001, 1.00002, 1.     ])

In [19]:
df = df.with_columns(
        pl.when(pl.col("clase_ternaria").is_in(["BAJA+1", "BAJA+2"])).then(1)
        .otherwise(0)
        .alias("clase_binaria")
    )

df

foto_mes,clase_ternaria,clase_peso,clase_peso_2,literal,clase_binaria
i64,str,f64,f64,i32,i32
202101,"""BAJA+1""",1.00001,1.0,1,1
202102,"""BAJA+2""",1.00002,1.0,1,1
202103,"""CONTINUA""",1.0,1.0,0,0
202104,"""BAJA+2""",1.00002,1.0,1,1
202105,"""BAJA+1""",1.00001,1.0,1,1
202106,"""BAJA+2""",1.00002,1.0,1,1
